In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# standard lib
import os, pwd, sys, json, yaml, atexit, tempfile, inspect
from pathlib import Path

# for data-science
import pandas as pd, numpy as np, quadfeather
from pyarrow import feather

# for plotting
import matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

# for cellular-data
import scprep, scanpy as sc, anndata as ad

In [3]:
from featherplot.utils import MockSingleCellData, AnnDataProcessor, QuadFeatherRenamer
from featherplot.utils import SeriesToChannel, DataFrameToMetadata

from featherplot.utils import collapse_user
from featherplot.deepscatter import Tileset

In [4]:
adata = ad.read_h5ad(os.path.expanduser('~/Downloads/adata_s1.h5'))

In [5]:
adata

AnnData object with n_obs × n_vars = 41253 × 29275
    obs: 'barcodes', 'batch', 'timepoint', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'log10_total_counts'
    var: 'ensembl_id', 'gene_symbol', 'feature_type', 'mito', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable_WT1', 'highly_variable_WT2', 'highly_variable_KO1', 'highly_variable_KO3', 'highly_variable'
    uns: 'batch_colors'
    obsm: 'X_pca', 'X_pca_hvg', 'X_phate', 'X_phate_hvg'
    layers: 'X_detected', 'X_magic', 'X_prenorm', 'X_scaled_normalised'

In [12]:
adata.obs.loc[:, 'conditions'] = adata.obs.batch

### HVGs

In [14]:
subsample_percentage = 0.5

total_cells = adata.shape[0]
sample_size = int(total_cells * subsample_percentage)

In [15]:
barcodes = pd.Series(adata.obs.index.to_numpy().astype(str), name='barcodes')
subsample = barcodes.sample(sample_size)

all_genes = pd.Series(adata.var.gene_symbol.to_numpy(), name='genes')
hvg_genes = pd.Series(adata.var[adata.var.highly_variable].gene_symbol.to_numpy(), name='hvg_genes')

hvg_idxs = hvg_genes.map(lambda g: adata.var.gene_symbol.tolist().index(g))

In [16]:
sdata = adata[subsample, hvg_idxs]
sdata.shape

(20626, 6274)

### Create Processor
> this will help us extract the embedding layer and the gene expression layer

In [17]:
pipe = AnnDataProcessor(sdata, 'X_phate_hvg', 'X_magic')

#### sidecars

Deepscatter calls additional columns `sidecars`, in our case those are the columns of gene expression. We place these values in `df_s`.

In [18]:
df_s = pipe.get_sidecars()
df_s.head()

,samd11,hes4,isg15,ensg00000242590,pusl1,aurkaip1,ccnl2,mrpl20-as1,mrpl20,vwa1,...,sry,rps4y1,zfy,tbl1y,usp9y,ddx3y,tmsb4y,ttty14,ensg00000273748,ensg00000276345
CCTTATAAACCACAGT_1_1,0.003283,0.118062,0.472036,0.027416,0.169887,1.030929,0.059460,0.109248,1.240861,0.016911,...,0.038441,1.540256,0.024484,0.080791,0.048672,0.069707,0.141571,0.805999,0.039596,0.245006
AAGAATTAAGCGCCAC_3,0.001689,0.113360,0.905589,0.065835,0.232847,1.322561,0.095525,0.142055,1.552950,0.018224,...,0.056539,1.575424,0.044290,0.126020,0.070170,0.133023,0.364901,1.001463,0.080280,0.387865
CCCAAGGGCCGGACCA_1_0,0.003540,0.130652,0.463194,0.027383,0.138941,1.004315,0.058813,0.097315,1.256352,0.018805,...,0.037742,1.418464,0.024481,0.077431,0.052463,0.061942,0.122234,0.679069,0.034625,0.200009
CCAGAGCGCAAAATTT_1_1,0.004077,0.139749,0.481334,0.024868,0.206931,1.081458,0.057473,0.119534,1.291117,0.021176,...,0.041151,1.581175,0.023106,0.078843,0.058081,0.080372,0.223433,0.839984,0.065277,0.259735
CACGATTCATCGCCAT_0_0,0.006096,0.169425,0.462624,0.069898,0.160462,1.016115,0.070059,0.105035,1.231948,0.019508,...,0.056821,1.633546,0.027202,0.082261,0.069610,0.071924,0.164898,0.758949,0.041318,0.234896


#### points

If our gene expression features are called `sidecars`, then what is the embedding layer called? Well it is just the "points" of the plot, so we will store these values in `df_p`.

**NOTE**: we also store conditions with `df_p` as whatever is in this DataFrame will be loaded by `Deepscatter` automatically. 

In [19]:
df_p = pipe.get_embedding(add_conditions=False)
df_p = df_p.join(pipe.adata.obs.conditions)
df_p.head()

,PHATE_HVG_1,PHATE_HVG_2,PHATE_HVG_3,conditions
CCTTATAAACCACAGT_1_1,-0.017727,-0.001601,0.000652,WT2
AAGAATTAAGCGCCAC_3,0.037121,0.000214,0.003753,KO3
CCCAAGGGCCGGACCA_1_0,-0.020872,-0.001243,0.001219,WT1
CCAGAGCGCAAAATTT_1_1,-0.011802,-0.001108,0.000120,WT2
CACGATTCATCGCCAT_0_0,-0.015922,-0.000356,0.000801,WT1


#### Combined
Now we combine `df_p` (points + condition) with `df_s` ("sidecars" i.e. gene expression). This is necessary as for script later on where we need to add the sidecars to already the `quadfeather`-ed (tiled) point data. 

In [20]:
df_all = df_p.join(df_s)
df_all.head()

,PHATE_HVG_1,PHATE_HVG_2,PHATE_HVG_3,conditions,samd11,hes4,isg15,ensg00000242590,pusl1,aurkaip1,...,sry,rps4y1,zfy,tbl1y,usp9y,ddx3y,tmsb4y,ttty14,ensg00000273748,ensg00000276345
CCTTATAAACCACAGT_1_1,-0.017727,-0.001601,0.000652,WT2,0.003283,0.118062,0.472036,0.027416,0.169887,1.030929,...,0.038441,1.540256,0.024484,0.080791,0.048672,0.069707,0.141571,0.805999,0.039596,0.245006
AAGAATTAAGCGCCAC_3,0.037121,0.000214,0.003753,KO3,0.001689,0.113360,0.905589,0.065835,0.232847,1.322561,...,0.056539,1.575424,0.044290,0.126020,0.070170,0.133023,0.364901,1.001463,0.080280,0.387865
CCCAAGGGCCGGACCA_1_0,-0.020872,-0.001243,0.001219,WT1,0.003540,0.130652,0.463194,0.027383,0.138941,1.004315,...,0.037742,1.418464,0.024481,0.077431,0.052463,0.061942,0.122234,0.679069,0.034625,0.200009
CCAGAGCGCAAAATTT_1_1,-0.011802,-0.001108,0.000120,WT2,0.004077,0.139749,0.481334,0.024868,0.206931,1.081458,...,0.041151,1.581175,0.023106,0.078843,0.058081,0.080372,0.223433,0.839984,0.065277,0.259735
CACGATTCATCGCCAT_0_0,-0.015922,-0.000356,0.000801,WT1,0.006096,0.169425,0.462624,0.069898,0.160462,1.016115,...,0.056821,1.633546,0.027202,0.082261,0.069610,0.071924,0.164898,0.758949,0.041318,0.234896


### QuadFeatherRenamer

Note: `quadfeather` and `deepscatter` are both under active development so things change all the time. At the moment `quadfeather` requires that `x` and `y` be in your DataFrame (it doesn't mind if `z` is there too). So this will handle the renaming of our columns.

In [23]:
df_all.index.name = df_p.index.name = df_s.index.name = 'barcodes'

In [24]:
qfr = QuadFeatherRenamer(df_all)

In [25]:
df_q, renamed = qfr.rename()
renamed

{'PHATE_HVG_1': 'x', 'PHATE_HVG_2': 'y', 'PHATE_HVG_3': 'z'}

### DataFrameToMetadata
 
`Deepscatter` is a really nice library; however, it also prefers to have its `plotAPI` method called with as much information as possible. This is a bit of a shame as it means that one you load your data with `deepscatter` you can't compute derived properties (e.g. domain of your data to scale the plot, check for what sidecars are availble, etc). 

The solution to this is simple. In order to have this information availble to us, we will just calculate it now (including which columns were renamed) and store it as metadata to use later

In [27]:
# NOTE: it is important to make sure that the index has a name!
df_q.index.name = 'barcodes'

#### unique columns

In [28]:
all_cols = df_q.columns.tolist()
uni_cols, col_cnts = np.unique(all_cols, return_counts=True)

for n, c in zip(uni_cols, col_cnts):
    if c == 1:
        continue

    i = 0
    for j, o in enumerate(all_cols):
        if o != n:
            continue
        all_cols[j] = f'{n}_{i}'
        i += 1

df_q.columns = all_cols

#### perpare metadata

In [29]:
d2m = DataFrameToMetadata(
    df_q, 
    include_index=True,
    embedding='x y z'.split(),
    alt_names={v:k for k,v in renamed.items()}
)

In [30]:
succ, fail = d2m.convert()
len(succ), len(fail)

(6279, 0)

In [31]:
meta = d2m.to_meta()

## Quadfeather Workflow

Now we can now run through thte `quadfeather` workflow right here in the notebook.

### 0) setup

In [32]:
# dump everything to downloads for easy access
outdir = os.path.expanduser('~/Downloads/seo_feather')
qf_dir = os.path.join(outdir, 'tiles')
if not os.path.isdir(qf_dir):
    os.makedirs(qf_dir)


p_file = os.path.join(outdir, 'points.parquet')
# NOTE: we never use s_file
# s_file = os.path.join(outdir, 'extras.parquet')
f_file = os.path.join(qf_dir, 'sidecars.feather')
m_file = os.path.join(outdir, 'meta.yml')


tile_size = 1000

### 1) create tiles

In [33]:
d2m.df.drop(columns=d2m.sidecars).to_parquet(p_file)
# d2m.df.drop(columns=d2m.embedding).to_parquet(s_file)

In [34]:
!quadfeather --files {p_file} \
             --tile_size {tile_size} \
             --destination {qf_dir}

### 2) make single file

In [35]:
feather.write_feather(d2m.df.drop(columns=d2m.embedding), f_file)

### 3) run `add_sidecars.py`

In [36]:
tileset = Tileset(Path(qf_dir))
tileset.add_sidecars(f_file, d2m.df.index.name)

note we copied `add_sidecars.py` so you can use it directly from this library

In [44]:
!featherplot add-sidecars --tileset {qf_dir}\
                         --sidecar {f_file} --key {d2m.df.index.name};

```sh
featherplot-py featherplot add-sidecars --help

Usage: featherplot add-sidecars 
[OPTIONS]
--tileset          PATH  Path to the tileset to add sidecars to.
--sidecar          PATH  Path to the new data to add to the tileset.
--key              TEXT  key to use for joining; must exist in both tables
--verbose  -v            Print verbose output.
--help                   Show this message and exit.
```

alternatively you can run the script form wherever you saved it

In [ ]:
!python3 add_sidecars.py --tileset {qf_dir}\
                         --sidecar {f_file} --key {d2m.df.index.name};

### 4) update metadata with directory

In [37]:
meta.keys()

dict_keys(['index', 'n_points', 'embedding', 'sidecars', 'columns_metadata', 'tiles_dir'])

In [38]:
# relative path to tiles
meta['tiles_dir'] = qf_dir.replace(outdir, '')
# full path to tiles
meta['full_path'] = collapse_user(qf_dir)

In [39]:
with open(m_file, 'w') as f:
    f.write(yaml.dump(meta))

### 5) cleanup